In [11]:
import pandas as pd
import io

# Sample data
data = """source_ip dest_ip dest_port BLOCK/ALLOW
0.0.0.0/0 10.55.163.0/24 ALL BLOCK
10.55.163.0/24 0.0.0.0/0 ALL ALLOW
0.0.0.0/0 10.55.163.141 443/tcp ALLOW
0.0.0.0/0 10.55.163.141 6443/tcp ALLOW
0.0.0.0/0 10.55.163.141 80/tcp ALLOW
"""


# Create a DataFrame using a custom parser function
def custom_parser(x):
    parts = x.split('/')
    if len(parts) == 1:
        return [parts[0], None]
    else:
        return parts


df = pd.read_csv(io.StringIO(data), delim_whitespace=True,
                 converters={'source_ip': custom_parser, 'dest_ip': custom_parser, 'dest_port': custom_parser})
df
# # Create new columns for the values before and after the slash
# df[['source_ip', 'source_port1', 'source_port2']] = pd.DataFrame(df['source_ip'].tolist(), index=df.index)
# df[['dest_ip', 'dest_port1', 'dest_port2']] = pd.DataFrame(df['dest_ip'].tolist(), index=df.index)
# 
# # Drop the original columns
# df = df.drop(['source_ip', 'dest_ip'], axis=1)
# 
# # Rename columns for better clarity
# df.columns = ['BLOCK_ALLOW', 'source_ip', 'source_port1', 'source_port2', 'dest_ip', 'dest_port1', 'dest_port2']
# 
# # Display the DataFrame
# print(df)


,source_ip,dest_ip,dest_port,BLOCK/ALLOW
0,"[0.0.0.0, 0]","[10.55.163.0, 24]","[ALL, None]",BLOCK
1,"[10.55.163.0, 24]","[0.0.0.0, 0]","[ALL, None]",ALLOW
2,"[0.0.0.0, 0]","[10.55.163.141, None]","[443, tcp]",ALLOW
3,"[0.0.0.0, 0]","[10.55.163.141, None]","[6443, tcp]",ALLOW
4,"[0.0.0.0, 0]","[10.55.163.141, None]","[80, tcp]",ALLOW


In [18]:
import pandas as pd
import io

# Sample data
data = """source_ip dest_ip dest_port BLOCK/ALLOW
0.0.0.0/0 10.55.163.0/24 ALL BLOCK
10.55.163.0/24 0.0.0.0/0 ALL ALLOW
0.0.0.0/0 10.55.163.141 443/tcp ALLOW
0.0.0.0/0 10.55.163.141 6443/tcp ALLOW
0.0.0.0/0 10.55.163.141 80/tcp ALLOW
"""

def custom_parser(x):
    parts = x.split('/')
    if len(parts) == 1:
        return pd.Series({'ip': parts[0], 'ip_range': None})
    else:
        return pd.Series({'ip': parts[0], 'ip_range': parts[1]})

def split_dest_port(x):
    parts = x.split('/')
    if len(parts) == 1:
        return pd.Series({'dest_port': parts[0], 'protocol': None})
    else:
        return pd.Series({'dest_port': parts[0], 'protocol': parts[1]})

df = pd.read_csv(io.StringIO(data), delim_whitespace=True,
                 converters={'source_ip': custom_parser, 'dest_ip': custom_parser, 'dest_port': split_dest_port})

# Split 'source_ip' and 'dest_ip' into two separate columns
df[['source_ip', 'source_ip_range']] = df['source_ip'].apply(pd.Series)
df[['dest_ip', 'dest_ip_range']] = df['dest_ip'].apply(pd.Series)
df[['dest_port', 'dest_port_type']] = df['dest_port'].apply(pd.Series)

# Drop the original columns
# df = df.drop(['source_ip', 'dest_ip'], axis=1)

# Display the modified DataFrame
df

,source_ip,dest_ip,dest_port,BLOCK/ALLOW,source_ip_range,dest_ip_range,dest_port_type
0,0.0.0.0,10.55.163.0,ALL,BLOCK,0,24,None
1,10.55.163.0,0.0.0.0,ALL,ALLOW,24,0,None
2,0.0.0.0,10.55.163.141,443,ALLOW,0,None,tcp
3,0.0.0.0,10.55.163.141,6443,ALLOW,0,None,tcp
4,0.0.0.0,10.55.163.141,80,ALLOW,0,None,tcp
